In [307]:
import sqlite3
from sqlite3 import Error

In [308]:
connection = sqlite3.connect('lib2')

cursor = connection.cursor()
cursor.execute(
    '''
    CREATE TABLE Book(
        id INT PRIMARY KEY NOT NULL,
        title TEXT,
        author TEXT,
        publish_year INT
    )
    '''
)

cursor.execute(
    '''
    INSERT INTO Book VALUES
    (1, 'Harry Potter and the Philosopher''s Stone', 'J. K. Rowling', 1997),
    (2, 'Методы решения задач в общем курсе физики, Механика', 'Корявов В.П.', 2007),
    (3, 'Batman: Gotham by Gaslight', 'Brian Augustyn, Mike Mignola', 1989),
    (4, 'Мёртвые души', 'Н. В. Гоголь', 1842),
    (5, 'The Running Man', 'Richard Bachman', 1982)
    '''
)


In [309]:
cursor.execute(
    '''
    CREATE TABLE Reader(
        id INT PRIMARY KEY NOT NULL,
        name TEXT
    )
    '''
)

cursor.execute(
    '''
    INSERT INTO Reader VALUES
    (1, 'V.V. Petrovich'),
    (2, 'Alexanderr'),
    (3, 'Arkadiy Svidrigaylov'),
    (4, 'Vitalya'),
    (5, 'A.S.Pushkin')
    '''
)


In [310]:
cursor.execute(
    '''
    CREATE TABLE Records(
        id INT PRIMARY KEY NOT NULL,
        reader_id INT REFERENCES Readers(id) NOT NULL,
        book_id INT REFERENCES Books(id) NOT NULL,
        taking_date TEXT,
        returning_date TEXT
    )
    '''
)

cursor.execute(
    '''
    INSERT INTO Records VALUES
    (1, 3, 4, '1842-02-14', '1842-07-01'),
    (2, 4, 4, '1996-10-26', 'NULL'),
    (3, 1, 2, '2006-06-13', '2006-06-14'),
    (4, 1, 1, '2006-06-14', '2006-06-15'),
    (5, 1, 3, '2006-06-15', '2006-07-20'),
    (6, 2, 5, '1985-10-11', '1987-12-25'),
    (7, 5, 3, '2001-03-15', 'NULL'),
    (8, 5, 3, '2002-09-19', '2002-09-25')
    '''
)

In [311]:
connection.commit()
connection.close()

Здесь под списком книг я понимается именно названия этих книг(хотя в принципе можно вывести полную информацию о книгах). Аналогично про читателей - только их записанные "имена".

In [312]:
def book_selecter():
    connection = sqlite3.connect("lib2")
    with connection:
        for row in connection.execute("SELECT title FROM Book"):
            print(row)
            
book_selecter()

("Harry Potter and the Philosopher's Stone",)
('Методы решения задач в общем курсе физики, Механика',)
('Batman: Gotham by Gaslight',)
('Мёртвые души',)
('The Running Man',)


In [313]:
def reader_selecter():
    connection = sqlite3.connect("lib2")
    with connection:
        for row in connection.execute("SELECT name FROM Reader"):
            print(row)

reader_selecter()

('V.V. Petrovich',)
('Alexanderr',)
('Arkadiy Svidrigaylov',)
('Vitalya',)
('A.S.Pushkin',)


Функция работает так - в неё поступает кортеж с информацией об пользователе/книге, она добавляет её в БД.

In [314]:
def add_book(book_info):
    connection = sqlite3.connect("lib2")
    with connection:
        connection.execute("INSERT OR IGNORE INTO Book VALUES(?, ?, ?, ?)", book_info)
        connection.commit()

add_book((6, 'The Veldt', 'Ray Bradbury', 1950))
add_book((7, 'KK', 'Ray Bradbury', 1950)) # Лишняя книга 

In [315]:
book_selecter()

("Harry Potter and the Philosopher's Stone",)
('Методы решения задач в общем курсе физики, Механика',)
('Batman: Gotham by Gaslight',)
('Мёртвые души',)
('The Running Man',)
('The Veldt',)
('KK',)


Здесь же сделаем функцию удаления книги (аналогом в обычной библиотеки могут быть списания, утери т.д.).

In [316]:
def delete_book(title):
    connection = sqlite3.connect("lib2")
    with connection:
        connection.execute("DELETE FROM Book WHERE title=?", (title,))
        connection.commit()

delete_book('KK')

In [317]:
book_selecter()

("Harry Potter and the Philosopher's Stone",)
('Методы решения задач в общем курсе физики, Механика',)
('Batman: Gotham by Gaslight',)
('Мёртвые души',)
('The Running Man',)
('The Veldt',)


In [318]:
def add_reader(reader_info):
    connection = sqlite3.connect("lib2")
    with connection:
        connection.execute("INSERT OR IGNORE INTO Reader VALUES(?, ?)", reader_info)
        connection.commit()

add_reader((6, 'Ilya Ilyich Oblomov'))

In [319]:
reader_selecter()

('V.V. Petrovich',)
('Alexanderr',)
('Arkadiy Svidrigaylov',)
('Vitalya',)
('A.S.Pushkin',)
('Ilya Ilyich Oblomov',)


In [320]:
def record_selecter():
    connection = sqlite3.connect("lib2")
    with connection:
        for row in connection.execute("SELECT * FROM Records"):
            print(row)
            
record_selecter()

(1, 3, 4, '1842-02-14', '1842-07-01')
(2, 4, 4, '1996-10-26', 'NULL')
(3, 1, 2, '2006-06-13', '2006-06-14')
(4, 1, 1, '2006-06-14', '2006-06-15')
(5, 1, 3, '2006-06-15', '2006-07-20')
(6, 2, 5, '1985-10-11', '1987-12-25')
(7, 5, 3, '2001-03-15', 'NULL')
(8, 5, 3, '2002-09-19', '2002-09-25')


Хотим выдавать книги:

In [329]:
def lend_book(record_info):
    connection = sqlite3.connect("lib2")
    with connection:
        connection.execute("INSERT OR IGNORE INTO Records VALUES(?, ?, ?, ?, ?)", record_info)
        connection.commit()
        
lend_book((9, 6, 6, '2022-03-23', '2022-04-06'))

In [330]:
record_selecter()

(1, 3, 4, '1842-02-14', '1842-07-01')
(2, 4, 4, '1996-10-26', 'NULL')
(3, 1, 2, '2006-06-13', '2006-06-14')
(4, 1, 1, '2006-06-14', '2006-06-15')
(5, 1, 3, '2006-06-15', '2006-07-20')
(6, 2, 5, '1985-10-11', '1987-12-25')
(7, 5, 3, '2001-03-15', 'NULL')
(8, 5, 3, '2002-09-19', '2002-09-25')
(9, 6, 6, '2022-03-23', '2022-04-06')


Хотим принимать книги:

In [331]:
def accept_record(record_info):
    connection = sqlite3.connect("lib2")
    with connection:
        connection.execute("INSERT OR IGNORE INTO Records VALUES(?, ?, ?, ?, ?)", record_info)
        connection.commit()
        
accept_record((10, 6, 6, '2022-03-25', 'NULL'))

In [332]:
record_selecter()

(1, 3, 4, '1842-02-14', '1842-07-01')
(2, 4, 4, '1996-10-26', 'NULL')
(3, 1, 2, '2006-06-13', '2006-06-14')
(4, 1, 1, '2006-06-14', '2006-06-15')
(5, 1, 3, '2006-06-15', '2006-07-20')
(6, 2, 5, '1985-10-11', '1987-12-25')
(7, 5, 3, '2001-03-15', 'NULL')
(8, 5, 3, '2002-09-19', '2002-09-25')
(9, 6, 6, '2022-03-23', '2022-04-06')
(10, 6, 6, '2022-03-25', 'NULL')
